In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass
from data_loader import prepare_ml_pipeline, MovieLensData, MovieLensDataset
from matrix_factor import BiasedMF, train_mf
from debiasing import train_debiasing_model, DebiasingModel
from dropoutnet import DeepCF, train_dropoutnet
from evaluator import ndcg_calc_base, ndcg_calc_dropout, ndcg_calc_debiased, ndcg_calc_sampled, ndcg_calc_dropout_sampled, ndcg_calc_debiased_sampled
from adversarial import Filter, Discriminator, train_adversarial

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def run_evaluation_pipeline(device: str = 'cuda' if torch.cuda.is_available() else 'cpu'):
    """Complete evaluation pipeline with DropoutNet and debiasing"""
    
    # Define evaluation parameters
    k_values = [15,30]
    

    dropoutnet_params = {
        'model_select': [800, 400],
        'rank_out': 200,
        'dropout_rate': 0.5,
        'batch_size': 1000,
        'n_scores_per_user': 2500,
        'data_batch_size': 1000,
        'max_data_per_step': 50000,
        'num_epochs': 1,
        'learning_rate': 0.001
    }
    
    #Params from https://github.com/Zziwei/Fairness-in-Cold-Start-Recommendation/blob/main/Scale/main.py
    debiasing_params = {
        'model_select': [100],
        'alpha': 4.0,
        'batch_size': 50,
        'num_epochs': 1,
        'reg': 0.00001
    }
    
    # 1. Load and prepare data
    print("Loading data...")
    ml_data, train_loader, valid_loader, test_loader = prepare_ml_pipeline(cold_start=True)
    #evaluator = RecommenderEvaluator(ml_data)
    
    # 2. Train and evaluate base model
    print("\nTraining base model...")
    base_model = BiasedMF(ml_data.n_users, ml_data.n_items).to(device)
    base_model = train_mf(model = base_model, train_loader= train_loader, val_loader= valid_loader, ml_data= ml_data, num_epochs=1, lr = .01)
    
    #base_ndcgs, base_prec, base_recall, final_mdg, mdg_anal = ndcg_calc_base(base_model, test_loader, ml_data, k_values=k_values)
    #print(f"Base NDCGS: {base_ndcgs}")
    #print(f"Final MDG: {mdg_anal['all']['mean']}, min10: {mdg_anal['bottom_10']['mean']}, min20: {mdg_anal['bottom_20']['mean']}, top10: {mdg_anal['top_10']['mean']}")


    #2.5 Debias the base model embeddings
    print("\nAdversarial Debiasing...")
    user_embeddings, user_bias, item_embeddings, item_bias, global_bias = base_model.get_all_embeddings()
    print("EMBEDDING")
    debias_filter = Filter(user_embeddings, user_embeddings.shape[0]).to(device)
    debias_adversary = Discriminator(debias_filter, user_bias, item_embeddings, item_bias, global_bias).to(device)
    debias_filter, debias_adversary = train_adversarial(train_loader, debias_filter, debias_adversary, user_embeddings, user_bias, item_embeddings, item_bias, global_bias)
    

    
    #3. Train and evaluate DropoutNet model
    print("\nTraining DropoutNet model...")
    dropoutnet = train_dropoutnet(
        ml_data=ml_data,
        base_model=base_model,
        val_loader=valid_loader,
        test_loader=test_loader,
        **dropoutnet_params,
        device=device
    )

    print("\nEvaluating DropoutNet model...")
    dropout_ndcgs, drop_mdg, drop_mdg_anal = ndcg_calc_dropout_sampled(base_model, dropoutnet, test_loader, ml_data, k_values = k_values)
    print(f"Dropout NDCGs {dropout_ndcgs}")
    print(drop_mdg_anal)
    print(f"Final MDG: {drop_mdg_anal['all']['mean']}, min10: {drop_mdg_anal['bottom_10']['mean']}, min20: {drop_mdg_anal['bottom_20']['mean']}, top10: {drop_mdg_anal['top_10']['mean']}")

    
    # 4. Train and evaluate debiasing model
    print("\nTraining debiasing model...")
    debiasing_model = train_debiasing_model(
        base_model=dropoutnet,
        original_mf=base_model,
        ml_data=ml_data,
        **debiasing_params,
        device=device
    )
    
    print("\nEvaluating debiased model...")
    debiased_ndcgs, debiased_prec, debiased_rec, debiased_mdg, debiased_mdg_anal = ndcg_calc_debiased_sampled(dropoutnet, base_model, debiasing_model,test_loader,ml_data,k_values=k_values)
    
    print(f"Debiased NDCGs {debiased_ndcgs}")
    print(f"Final MDG: {debiased_mdg_anal['all']['mean']}, min10: {debiased_mdg_anal['bottom_10']['mean']}, min20: {debiased_mdg_anal['bottom_20']['mean']}, top10: {debiased_mdg_anal['top_10']['mean']}")
    
   
    return 0


In [ ]:
test = run_evaluation_pipeline()